In [ ]:
import numpy as np
import pandas as pd
from dtaidistance import dtw
from scipy.spatial.distance import squareform
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Supongamos que tienes tus datos en df_agg
# df_agg debe tener columnas: ['periodo', 'product_id', 'tn']

def prepare_series_matrix(df_agg):
    """
    Convierte el DataFrame en una matriz donde cada fila es una serie temporal
    """
    # Pivot para tener series como filas
    pivot_df = df_agg.pivot(index='product_id', columns='periodo', values='tn')
    
    # Rellenar valores faltantes si los hay
    pivot_df = pivot_df.fillna(method='ffill').fillna(method='bfill').fillna(0)
    
    print(f"Matriz de series: {pivot_df.shape}")
    print(f"Productos: {len(pivot_df)}")
    print(f"Períodos: {len(pivot_df.columns)}")
    
    return pivot_df

def calculate_dtw_matrix(series_matrix, normalize=True, window=None, max_series=None):
    """
    Calcula la matriz de distancias DTW
    
    Parameters:
    - series_matrix: DataFrame con series como filas
    - normalize: Si normalizar las series antes del cálculo
    - window: Ventana de restricción para DTW (acelera el cálculo)
    - max_series: Límite máximo de series para procesar (para pruebas)
    """
    
    # Limitar series si se especifica (para pruebas)
    if max_series and max_series < len(series_matrix):
        series_matrix = series_matrix.iloc[:max_series]
        print(f"Limitando a {max_series} series para prueba")
    
    # Convertir a numpy array
    series_array = series_matrix.values
    
    # Normalizar si se requiere
    if normalize:
        scaler = StandardScaler()
        series_array = scaler.fit_transform(series_array.T).T
        print("Series normalizadas")
    
    n_series = len(series_array)
    print(f"Calculando DTW para {n_series} series...")
    print(f"Total de comparaciones: {n_series * (n_series - 1) // 2}")
    
    # Inicializar matriz de distancias
    distance_matrix = np.zeros((n_series, n_series))
    
    # Calcular DTW con barra de progreso
    for i in tqdm(range(n_series), desc="Procesando series"):
        for j in range(i + 1, n_series):
            try:
                if window:
                    dist = dtw.distance(series_array[i], series_array[j], window=window)
                else:
                    dist = dtw.distance(series_array[i], series_array[j])
                
                distance_matrix[i, j] = dist
                distance_matrix[j, i] = dist
            except Exception as e:
                print(f"Error calculando DTW entre serie {i} y {j}: {e}")
                distance_matrix[i, j] = np.inf
                distance_matrix[j, i] = np.inf
    
    # Crear DataFrame con índices de productos
    distance_df = pd.DataFrame(
        distance_matrix,
        index=series_matrix.index,
        columns=series_matrix.index
    )
    
    return distance_df

def analyze_dtw_results(distance_df, top_n=10):
    """
    Analiza los resultados de DTW
    """
    print("=== ANÁLISIS DE RESULTADOS DTW ===")
    
    # Estadísticas básicas
    mask = np.triu(np.ones_like(distance_df.values, dtype=bool), k=1)
    upper_triangle = distance_df.values[mask]
    upper_triangle = upper_triangle[upper_triangle != np.inf]
    
    print(f"Estadísticas de distancias DTW:")
    print(f"  Media: {np.mean(upper_triangle):.2f}")
    print(f"  Mediana: {np.median(upper_triangle):.2f}")
    print(f"  Desviación estándar: {np.std(upper_triangle):.2f}")
    print(f"  Mínimo: {np.min(upper_triangle):.2f}")
    print(f"  Máximo: {np.max(upper_triangle):.2f}")
    
    # Pares más similares (menor distancia DTW)
    print(f"\n=== TOP {top_n} PARES MÁS SIMILARES ===")
    
    # Obtener índices del triángulo superior
    indices = np.triu_indices_from(distance_df.values, k=1)
    distances = distance_df.values[indices]
    
    # Crear DataFrame con pares y distancias
    pairs_df = pd.DataFrame({
        'product_1': distance_df.index[indices[0]],
        'product_2': distance_df.index[indices[1]],
        'dtw_distance': distances
    })
    
    # Filtrar infinitos
    pairs_df = pairs_df[pairs_df['dtw_distance'] != np.inf]
    
    # Ordenar por distancia
    similar_pairs = pairs_df.nsmallest(top_n, 'dtw_distance')
    print(similar_pairs.to_string(index=False))
    
    # Pares más diferentes (mayor distancia DTW)
    print(f"\n=== TOP {top_n} PARES MÁS DIFERENTES ===")
    different_pairs = pairs_df.nlargest(top_n, 'dtw_distance')
    print(different_pairs.to_string(index=False))
    
    return pairs_df

def visualize_dtw_results(distance_df, pairs_df, series_matrix):
    """
    Visualiza los resultados de DTW
    """
    # 1. Heatmap de matriz de distancias (muestra reducida)
    plt.figure(figsize=(12, 10))
    
    # Tomar una muestra para visualización
    sample_size = min(50, len(distance_df))
    sample_matrix = distance_df.iloc[:sample_size, :sample_size]
    
    plt.subplot(2, 2, 1)
    sns.heatmap(sample_matrix, annot=False, cmap='viridis', 
                square=True, cbar_kws={'label': 'Distancia DTW'})
    plt.title(f'Matriz de distancias DTW (muestra de {sample_size} series)')
    plt.xlabel('Product ID')
    plt.ylabel('Product ID')
    
    # 2. Histograma de distancias
    plt.subplot(2, 2, 2)
    valid_distances = pairs_df['dtw_distance'][pairs_df['dtw_distance'] != np.inf]
    plt.hist(valid_distances, bins=50, alpha=0.7, edgecolor='black')
    plt.xlabel('Distancia DTW')
    plt.ylabel('Frecuencia')
    plt.title('Distribución de distancias DTW')
    plt.grid(True, alpha=0.3)
    
    # 3. Ejemplo de par más similar
    plt.subplot(2, 2, 3)
    most_similar = pairs_df.loc[pairs_df['dtw_distance'].idxmin()]
    prod1, prod2 = most_similar['product_1'], most_similar['product_2']
    
    plt.plot(series_matrix.loc[prod1], label=f'Producto {prod1}', marker='o')
    plt.plot(series_matrix.loc[prod2], label=f'Producto {prod2}', marker='s')
    plt.title(f'Par más similar (DTW: {most_similar["dtw_distance"]:.2f})')
    plt.xlabel('Período')
    plt.ylabel('Valor')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 4. Ejemplo de par más diferente
    plt.subplot(2, 2, 4)
    most_different = pairs_df.loc[pairs_df['dtw_distance'].idxmax()]
    prod1, prod2 = most_different['product_1'], most_different['product_2']
    
    plt.plot(series_matrix.loc[prod1], label=f'Producto {prod1}', marker='o')
    plt.plot(series_matrix.loc[prod2], label=f'Producto {prod2}', marker='s')
    plt.title(f'Par más diferente (DTW: {most_different["dtw_distance"]:.2f})')
    plt.xlabel('Período')
    plt.ylabel('Valor')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# EJEMPLO DE USO COMPLETO:
"""
# 1. Preparar datos
series_matrix = prepare_series_matrix(df_agg)

# 2. Calcular DTW (prueba con pocas series primero)
distance_df = calculate_dtw_matrix(
    series_matrix, 
    normalize=True,     # Normalizar para comparar patrones
    window=10,          # Ventana para acelerar (opcional)
    max_series=100      # Límite para prueba (remover para todas)
)

# 3. Analizar resultados
pairs_df = analyze_dtw_results(distance_df, top_n=10)

# 4. Visualizar
visualize_dtw_results(distance_df, pairs_df, series_matrix)

# 5. Guardar resultados
distance_df.to_csv('dtw_distance_matrix.csv')
pairs_df.to_csv('dtw_pairs_analysis.csv', index=False)
"""

# Para el procesamiento completo de 1230 series:
def process_full_dataset():
    """
    Proceso optimizado para el dataset completo
    """
    print("=== PROCESAMIENTO COMPLETO DE 1230 SERIES ===")
    pathdata='data/'
    filename = 'sell-in.txt'

    # Unir ruta completa
    filepath = os.path.join(pathdata, filename)
    sell = pd.read_csv(filepath, sep="\t")
    sell=sell[sell['periodo'] <= 201910]
    
    df_agg = sell.groupby(['periodo', 'product_id'], as_index=False).agg({
    'tn': 'sum'
    })
    
    # Preparar datos
    series_matrix = prepare_series_matrix(df_agg)
    
    # Calcular DTW con optimizaciones
    distance_df = calculate_dtw_matrix(
        series_matrix,
        normalize=True,
        window=18,  # Ventana de la mitad de la serie (36/2)
        max_series=None  # Procesar todas las series
    )
    
    # Analizar
    #pairs_df = analyze_dtw_results(distance_df, top_n=20)
    
    # Guardar resultados
    distance_df.to_csv('dtw_matrix_octubre19.csv')
    #pairs_df.to_csv('dtw_pairs_full.csv', index=False)
    
    print("Resultados guardados en 'dtw_matrix_full.csv' y 'dtw_pairs_full.csv'")
    pairs_df=0
    return distance_df, pairs_df



In [ ]:
distance_df, pairs_df = process_full_dataset()